# Conic Programming
### Arpit Bhatia

This tutorial is aimed at providing a simplistic introduction to conic programming using JuMP.

# What is a Cone?
A subset $C$ of a vector space $V$ is a cone if $\forall x \in C$ and positive scalars $\alpha$, 
the product $\alpha x \in C$. A cone C is a convex cone if $\alpha x + \beta y \in C$, 
for any positive scalars $\alpha, \beta$, and any $x, y \in C$.

# Conic Programming
Conic programming problems are convex optimization problems in which a convex function is minimized
over the intersection of an affine subspace and a convex cone. 
An example of a conic-form minimization problems, in the primal form is:

$$
\begin{align}
& \min_{x \in \mathbb{R}^n} & a_0^T x + b_0 \\
& \;\;\text{s.t.} & A_i x + b_i & \in \mathcal{C}_i & i = 1 \ldots m
\end{align}
$$

The corresponding dual problem is:

$$
\begin{align}
& \max_{y_1, \ldots, y_m} & -\sum_{i=1}^m b_i^T y_i + b_0 \\
& \;\;\text{s.t.} & a_0 - \sum_{i=1}^m A_i^T y_i & = 0 \\
& & y_i & \in \mathcal{C}_i^* & i = 1 \ldots m
\end{align}
$$

where each $\mathcal{C}_i$ is a closed convex cone and $\mathcal{C}_i^*$ is its dual cone.

# Some of the Types of Cones Supported by JuMP

In [1]:
using JuMP
using ECOS
using CSDP

By this point we have used quite a few different solvers. 
To find out all the different solvers and their supported problem types, check out the 
[solver table](http://www.juliaopt.org/JuMP.jl/v0.19.0/installation/#Getting-Solvers-1) in the docs.

## Second-Order Cone
The Second-Order Cone (or Lorenz Cone) of dimension $n$ is of the form:

$$
Q^n = \{ (t,x) \in \mathbb{R}^\mbox{n} : t \ge ||x||_2 \}
$$

A Second-Order Cone rotated by $\pi/4$ in the $(x_1,x_2)$ plane is called a Rotated Second-Order Cone.
It is of the form:

$$
Q_r^n = \{ (t,u,x) \in \mathbb{R}^\mbox{n} : 2tu \ge ||x||_2^2, t,u \ge 0 \}
$$

These cones are represented in JuMP using the MOI sets `SecondOrderCone` and `RotatedSecondOrderCone`.

### Example: Euclidean Projection on a Hyperplane
For a given point $u_{0}$ and a set $K$, we refer to any point $u \in K$ 
which is closest to $u_{0}$ as a projection of $u_{0}$ on $K$. 
The projection of a point $u_{0}$ on a hyperplane $K = \{u | p' \cdot u = q\}$ is given by

$$
\begin{align*}
\min && ||u - u_{0}|| \\
s.t. && p' \cdot u = q 
\end{align*}
$$

In [2]:
u0 = rand(10)
p = rand(10)
q = rand();

We can model the above problem as the following conic program:

$$
\begin{align*}
\min t \\
\text { s.t. }  p' \cdot u = q \\
(t, u - u_{0}) \in Q^{n+1}
\end{align*}
$$

On comparing this with the primal form of a conic problem we saw above,

$$
\begin{align*}
x = (t , u) \\
a_0 = e_1 \\
b_0 = 0 \\
A_1 = (0, p) \\
b_1 = -q \\
C_1 = \mathbb{R}_- \\
A_2 = 1 \\
b_2 = -(0, u_0) \\
C_2 = Q^{n+1} 
\end{align*}
$$

Thus, we can obtain the dual problem as:

$$
\begin{align*}
\max q  y_1 + (0, u_0)^T y_2 \\
\text { s.t. } e_1 - (0,p)^T y_1 - y_2 = 0 \\
y_1 \in ? \\
y_2 \in Q^{n+1} 
\end{align*}
$$

In [3]:
model = Model(with_optimizer(ECOS.Optimizer, printlevel = 0))
@variable(model, u[1:10])
@variable(model, t)
@objective(model, Min, t)
@constraint(model, [t, (u - u0)...] in SecondOrderCone())
@constraint(model, u' * p == q) 
optimize!(model)


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +1e+01  4e-01  3e-04  1e+00  6e+00    ---    ---    1  1  - |  -  - 
 1  +3.668e-01  +3.749e-01  +1e+00  7e-02  3e-05  1e-01  7e-01  0.8882  2e-03   2  2  2 |  0  0
 2  +1.514e+00  +1.542e+00  +7e-02  5e-03  2e-06  4e-02  5e-02  0.9716  5e-02   2  2  2 |  0  0
 3  +1.553e+00  +1.554e+00  +8e-04  5e-05  2e-08  4e-04  6e-04  0.9890  1e-04   1  1  1 |  0  0
 4  +1.554e+00  +1.554e+00  +9e-06  6e-07  2e-10  5e-06  7e-06  0.9890  1e-04   1  1  1 |  0  0
 5  +1.554e+00  +1.554e+00  +1e-07  7e-09  2e-12  5e-08  7e-08  0.9890  1e-04   1  1  1 |  0  0
 6  +1.554e+00  +1.554e+00  +1e-09  7e-11  3e-14  6e-10  8e-10  0.9890  1e-04   2  1  1 |  0  0

OPTIMAL (within feastol=7.3e-11, reltol=6.8e-10, abstol=1.1e-09).
Runtime: 0.000239 seconds.



In [4]:
@show value.(u)

value.(u) = [-0.502476, 0.111571, 0.525742, -0.0222537, 0.458049, -0.320117, -0.145291, 0.22761, 0.734226, -0.171106]


10-element Array{Float64,1}:
 -0.502476312823145   
  0.11157110806521377 
  0.5257421149175829  
 -0.022253706832804972
  0.4580490946464344  
 -0.3201172769617206  
 -0.14529055883392586 
  0.2276099498497713  
  0.7342259973253014  
 -0.17110609590022974 

We can also have an equivalent formulation using a Rotated Second-Order Cone:

$$
\begin{align*}
\min t \\
\text { s.t. }  p' \cdot u = q \\
(t, 1/2, u - u_{0})\in Q_r^{n+2}
\end{align*}
$$

In [5]:
model = Model(with_optimizer(ECOS.Optimizer, printlevel = 0))
@variable(model, u[1:10])
@variable(model, t)
@objective(model, Min, t)
@constraint(model, [t, 0.5, (u - u0)...] in RotatedSecondOrderCone())
@constraint(model, u' * p == q) 
optimize!(model)


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -3.376e-02  +9e+00  4e-01  2e-02  1e+00  5e+00    ---    ---    1  1  - |  -  - 
 1  +9.545e-02  +1.014e-01  +1e+00  6e-02  2e-03  1e-01  6e-01  0.8880  1e-03   1  1  1 |  0  0
 2  +1.529e+00  +2.307e+00  +7e-01  2e-01  5e-03  1e+00  4e-01  0.6251  5e-01   2  2  2 |  0  0
 3  +2.308e+00  +2.341e+00  +1e-02  4e-03  9e-05  4e-02  7e-03  0.9828  1e-04   1  1  1 |  0  0
 4  +2.371e+00  +2.380e+00  +2e-03  8e-04  2e-05  1e-02  1e-03  0.8362  1e-02   2  1  1 |  0  0
 5  +2.400e+00  +2.409e+00  +7e-04  4e-04  9e-06  9e-03  5e-04  0.8065  3e-01   2  1  2 |  0  0
 6  +2.410e+00  +2.412e+00  +1e-04  8e-05  2e-06  2e-03  9e-05  0.8260  1e-02   2  1  1 |  0  0
 7  +2.414e+00  +2.414e+00  +2e-05  1e-05  3e-07  3e-04  2e-05  0.8963  1e-01   2  2  2 |  0  0
 8  +2.414e+00  +2.414e+00  +4e-06  3e-06  5e-

In [6]:
@show value.(u)

value.(u) = [-0.502476, 0.111571, 0.525742, -0.0222537, 0.458049, -0.320117, -0.145291, 0.22761, 0.734226, -0.171106]


10-element Array{Float64,1}:
 -0.5024763128681569 
  0.11157110804418605
  0.5257421149179967 
 -0.02225370686805626
  0.4580490946426287 
 -0.32011727702240245
 -0.14529055884477976
  0.22760994983929111
  0.7342259973254041 
 -0.17110609547400837

The difference here is that the objective in the case of the Second-Order Cone is $||u - u_{0}||_2$,
while in the case of a Rotated Second-Order Cone is $||u - u_{0}||_2^2$.
However, the value of x is the same for both.

## Exponential Cone

An Exponential Cone is a set of the form:

$$
K_{exp} = \{ (x,y,z) \in \mathbb{R}^3 : y \exp (x/y) \le z, y > 0 \}
$$

It is represented in JuMP using the MOI set `ExponentialCone`.

### Example: Entropy Maximization
As the name suggests, the entropy maximization problem consists of maximizing the entropy function,
$H(x) = -x\log{x}$ subject to linear inequality constraints.

$$
\begin{align*}
\max - \sum_{i=1}^n x_i \log x_i \\
\text { s.t. } \mathbf{1}' x = 1 \\
Ax \leq b
\end{align*}
$$

We can model this problem using an exponential cone by using the following transformation:

$$
t\leq -x\log{x} \iff t\leq x\log(1/x)  \iff (1, x, t) \in K_{exp}
$$

Thus, our problem becomes,

$$
\begin{align*}
\max 1^Tt \\
\text { s.t. } Ax \leq b \\
1^T x = 1 \\
(1, x_i, t_i) \in K_{exp} && \forall i = 1 \ldots n
\end{align*}
$$

In [7]:
# Cannot use the exponential cone directly in JuMP, hence we import MOI to specify the set.
using MathOptInterface
const MOI = MathOptInterface

n = 15;
m = 10;
A = randn(m, n); 
b = rand(m, 1);

model = Model(with_optimizer(ECOS.Optimizer, printlevel = 0))
@variable(model, t[1:n])
@variable(model, x[1:n])
@objective(model, Max, sum(t))
@constraint(model, sum(x) == 1)
@constraint(model, A * x .<= b )
@constraint(model, con[i = 1:n], [1, x[i], t[i]] in MOI.ExponentialCone())

optimize!(model);


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  +5.090e+00  +6e+01  1e+00  8e-01  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  +3.364e+01  +2.234e+02  +2e+01  2e+01  2e+01  1e+02  4e-01  0.6351  7e-02   1  1  1 |  0  0
 2  -3.098e+01  +1.210e+01  +1e+01  8e-01  6e-01  4e+01  3e-01  0.7833  5e-01   1  1  1 |  0  1
 3  -1.862e+02  +4.850e+01  +4e+00  1e+00  1e+00  2e+02  1e-01  0.7253  1e-01   1  1  1 |  2  1
 4  -9.704e+02  +1.044e+02  +2e+00  2e+00  1e+00  1e+03  4e-02  0.7833  2e-01   1  1  1 |  3  1
 5  -4.348e+03  +1.742e+02  +4e-01  2e+00  1e+00  5e+03  8e-03  0.7833  1e-02   1  1  1 |  1  1
 6  -1.759e+04  -4.929e+01  +8e-02  2e+00  1e+00  2e+04  2e-03  0.7833  1e-02   2  1  1 |  1  1
 7  -7.075e+04  -2.289e+03  +2e-02  2e+00  1e+00  7e+04  4e-04  0.7833  1e-02   2  1  1 |  1  1
 8  -2.654e+05  -1.435e+04  +4e-03  2e+00  1e+

In [8]:
objective_value(model)

2.5151666024837517e10

## Positive Semidefinite Cone
The set of Positive Semidefinite Matrices of dimension $n$ form a cone in $\mathbb{R}^n$.
We write this set mathematically as

$$
\mathcal{S}_{+}^n = \{ X \in \mathcal{S}^n \mid z^T X z \geq 0, \: \forall z\in \mathbb{R}^n \}.
$$

A PSD cone is represented in JuMP using the MOI sets 
`PositiveSemidefiniteConeTriangle` (for upper triangle of a PSD matrix) and
`PositiveSemidefiniteConeSquare` (for a complete PSD matrix). 
However, it is prefferable to use the `PSDCone` shortcut as illustrated below.

### Example: Largest Eigenvalue of a Symmetric Matrix
Suppose $A$ has eigenvalues $\lambda_{1} \geq \lambda_{2} \ldots \geq \lambda_{n}$. 
Then the matrix $t I-A$ has eigenvalues $t-\lambda_{1}, t-\lambda_{2}, \ldots, t-\lambda_{n}$. 
Note that $t I-A$ is PSD exactly when all these eigenvalues are non-negative, 
and this happens for values $t \geq \lambda_{1} .$ 
Thus, we can model the problem of finding the largest eigenvalue of a symmetric matrix as:

$$
\begin{align*}
\lambda_{1} = \max t \\
\text { s.t. } t I-A \succeq 0
\end{align*}
$$

In [9]:
using LinearAlgebra

A = [3 2 4;
     2 0 2;
     4 2 3]

model = Model(with_optimizer(CSDP.Optimizer, printlevel = 0))
@variable(model, t)
@objective(model, Min, t)
@constraint(model, t .* Matrix{Float64}(I, 3, 3) - A in PSDCone())

optimize!(model)

In [10]:
objective_value(model)

8.000000000000036

# Other Cones and Functions
For other cones supported by JuMP, check out the 
[MathOptInterface Manual](http://www.juliaopt.org/MathOptInterface.jl/dev/apimanual/#Standard-form-problem-1).
A good resource for learning more about functions which can be modelled using cones is the 
[MOSEK Modeling Cookbook](https://docs.mosek.com/modeling-cookbook/index.html).